<a href="https://colab.research.google.com/github/ShadiFarzankia/Human-Value-Detection-with-NLP/blob/master/Human_Value_Detection_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Human Value Detection


### Required Libraries


In [1]:
import json
import pandas as pd
import os
from sklearn.metrics import f1_score

### Loading the Data

In [ ]:
#Loading the Values
with open('values.json') as value_file:
   values_data = json.load(value_file)

print(values_data)

In [3]:
values_data['values']

[{'alternativeName': 'Creativity/Imagination',
  'name': 'Be creative',
  'id': 'SDT1',
  'level2': 'Self-direction: thought',
  'level3': ['Openness to change'],
  'level4a': ['Personal focus'],
  'level4b': ['Growth, Anxiety-free'],
  'descriptions': ['allowing for more creativity or imagination',
   'being more creative',
   'fostering creativity',
   'promoting imagination']},
 {'alternativeName': 'Curious/Interested',
  'name': 'Be curious',
  'id': 'SDT2',
  'level2': 'Self-direction: thought',
  'level3': ['Openness to change'],
  'level4a': ['Personal focus'],
  'level4b': ['Growth, Anxiety-free'],
  'descriptions': ['being the more interesting option',
   'fostering curiosity',
   'making people more keen to learn',
   'promoting discoveries',
   'sparking interest']},
 {'alternativeName': 'Freedom of thought',
  'name': 'Have freedom of thought',
  'id': 'SDT3',
  'level2': 'Self-direction: thought',
  'level3': ['Openness to change'],
  'level4a': ['Personal focus'],
  'leve

### Extrating Values

In [4]:
values = { "1":set(), "2":set(), "3":set(), "4a":set(), "4b":set() }
levels = ["1", "2", "3", "4a", "4b"]
num_levels = len(levels)

In [5]:
for value in values_data["values"]:
  values["1"].add(value["name"])
  values["2"].add(value["level2"])
  for valueLevel3 in value["level3"]:
    values["3"].add(valueLevel3)
  for valueLevel4a in value["level4a"]:
      values["4a"].add(valueLevel4a)
  for valueLevel4b in value["level4b"]:
      values["4b"].add(valueLevel4b)

values["1"] = sorted(values["1"])
values["2"] = sorted(values["2"])
values["3"] = sorted(values["3"])
values["4a"] = sorted(values["4a"])
values["4b"] = sorted(values["4b"])

In [6]:
values

{'1': ['Be ambitious',
  'Be behaving properly',
  'Be broadminded',
  'Be capable',
  'Be choosing own goals',
  'Be compliant',
  'Be courageous',
  'Be creative',
  'Be curious',
  'Be daring',
  'Be forgiving',
  'Be helpful',
  'Be holding religious faith',
  'Be honest',
  'Be honoring elders',
  'Be humble',
  'Be independent',
  'Be intellectual',
  'Be just',
  'Be logical',
  'Be loving',
  'Be neat and tidy',
  'Be polite',
  'Be protecting the environment',
  'Be respecting traditions',
  'Be responsible',
  'Be self-disciplined',
  'Have a comfortable life',
  'Have a good reputation',
  'Have a safe country',
  'Have a sense of belonging',
  'Have a stable society',
  'Have a varied life',
  'Have a world at peace',
  'Have a world of beauty',
  'Have an exciting life',
  'Have an objective view',
  'Have equality',
  'Have freedom of action',
  'Have freedom of thought',
  'Have good health',
  'Have harmony with nature',
  'Have influence',
  'Have life accepted as is',

In [7]:
#Loading the Arguments
dataframe = pd.read_csv('arguments.tsv', encoding='utf-8', sep='\t', header=0)

print(dataframe)

     Argument ID   Part  Usage                      Conclusion       Stance  \
0         A01001    usa  train  Entrapment should be legalized  in favor of   
1         A01002    usa  train     We should ban human cloning  in favor of   
2         A01003    usa  train      We should abandon marriage      against   
3         A01004    usa  train       We should ban naturopathy      against   
4         A01005    usa  train         We should ban fast food  in favor of   
...          ...    ...    ...                             ...          ...   
5265      D27096  india   test    Nepotism exists in Bollywood      against   
5266      D27097  india   test    Nepotism exists in Bollywood  in favor of   
5267      D27098  india   test         India is safe for women  in favor of   
5268      D27099  india   test         India is safe for women  in favor of   
5269      D27100  india   test         India is safe for women      against   

                                                Pre

In [8]:
#Oberserving the Dataset
dataframe.head(5)

,Argument ID,Part,Usage,Conclusion,Stance,Premise
0,A01001,usa,train,Entrapment should be legalized,in favor of,if entrapment can serve to more easily capture...
1,A01002,usa,train,We should ban human cloning,in favor of,we should ban human cloning as it will only ca...
2,A01003,usa,train,We should abandon marriage,against,marriage is the ultimate commitment to someone...
3,A01004,usa,train,We should ban naturopathy,against,it provides a useful income for some people
4,A01005,usa,train,We should ban fast food,in favor of,fast food should be banned because it is reall...


### Splitting the Arguments


In [18]:
'''train_arguments = dataframe.loc[dataframe['Usage'] == 'train'].drop(['Usage'], axis=1).reset_index(drop=True)
valid_arguments = dataframe.loc[dataframe['Usage'] == 'validation'].drop(['Usage'], axis=1).reset_index(drop=True)
test_arguments = dataframe.loc[dataframe['Usage'] == 'test'].drop(['Usage'], axis=1).reset_index(drop=True)'''

def split_arguments(df_arguments):
    """Splits `DataFrame` by column `Usage` into `train`-, `validation`-, and `test`-arguments"""
    train_arguments = df_arguments.loc[df_arguments['Usage'] == 'train'].drop(['Usage'], axis=1).reset_index(drop=True)
    valid_arguments = df_arguments.loc[df_arguments['Usage'] == 'validation'].drop(['Usage'], axis=1).reset_index(drop=True)
    test_arguments = df_arguments.loc[df_arguments['Usage'] == 'test'].drop(['Usage'], axis=1).reset_index(drop=True)

    return train_arguments, valid_arguments, test_arguments

In [19]:
'''print('Training Size:', train_arguments.shape)
print('Validation Size:', valid_arguments.shape)
print('Test Size:', test_arguments.shape)'''

"print('Training Size:', train_arguments.shape)\nprint('Validation Size:', valid_arguments.shape)\nprint('Test Size:', test_arguments.shape)"

In [11]:
'''train_arguments.head(5)'''

,Argument ID,Part,Conclusion,Stance,Premise
0,A01001,usa,Entrapment should be legalized,in favor of,if entrapment can serve to more easily capture...
1,A01002,usa,We should ban human cloning,in favor of,we should ban human cloning as it will only ca...
2,A01003,usa,We should abandon marriage,against,marriage is the ultimate commitment to someone...
3,A01004,usa,We should ban naturopathy,against,it provides a useful income for some people
4,A01005,usa,We should ban fast food,in favor of,fast food should be banned because it is reall...


In [12]:
'''train_arguments['Stance'].value_counts()'''

in favor of    2208
against        2032
Name: Stance, dtype: int64

# BERT Library

In [13]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 74.8 MB/s eta 0:00:00


In [16]:
#Loading Labels
def load_labels_from_tsv(filepath, label_order):
  dataframe = pd.read_csv(filepath, encoding='utf-8', sep='\t', header=0)
  dataframe = dataframe[['Argument ID'] + label_order]
  return dataframe

In [20]:
def combine_columns(df_arguments, df_labels):
    """Combines the two `DataFrames` on column `Argument ID`"""
    return pd.merge(df_arguments, df_labels, on='Argument ID')

In [22]:
def load_arguments_from_tsv(filepath, default_usage='test'):
  dataframe = pd.read_csv(filepath, encoding='utf-8', sep='\t', header=0)
  dataframe['Usage'] = [default_usage] * len(dataframe)
  return dataframe

In [25]:
# format dataset
df_train_all = []
df_valid_all = []
df_arguments = load_arguments_from_tsv('arguments.tsv', default_usage='train')

for i in range(num_levels):
  label_filepath = os.path.join('labels-level{}.tsv'.format(levels[i]))
  # read labels from .tsv file
  df_labels = load_labels_from_tsv(label_filepath, values[levels[i]])
  # join arguments and labels
  df_full_level = combine_columns(df_arguments, df_labels)
  # split dataframe by usage
  train_arguments, valid_arguments, _ = split_arguments(df_full_level)
  df_train_all.append(train_arguments)
  df_valid_all.append(valid_arguments)

In [14]:
# import the library
import torch
from sklearn.metrics import f1_score

import numpy as np

from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          PreTrainedModel, BertModel, BertForSequenceClassification,
                          TrainingArguments, Trainer)

In [29]:
train_bert_model(df_train_all[i], os.path.join('', 'bert_train_level{}'.format(levels[i])),
                                 values[levels[i]])

NameError: ignored

In [15]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

NameError: ignored

In [40]:
def accuracy_thresh(y_pred, y_true, thresh=0.5, sigmoid=True):
    """Compute accuracy of predictions"""
    y_pred = torch.from_numpy(y_pred)
    y_true = torch.from_numpy(y_true)
    if sigmoid:
        y_pred = y_pred.sigmoid()

    return ((y_pred > thresh) == y_true.bool()).float().mean().item()


def f1_score_per_label(y_pred, y_true, value_classes, thresh=0.5, sigmoid=True):
    """Compute label-wise and averaged F1-scores"""
    y_pred = torch.from_numpy(y_pred)
    y_true = torch.from_numpy(y_true)
    if sigmoid:
        y_pred = y_pred.sigmoid()

    y_true = y_true.bool().numpy()
    y_pred = (y_pred > thresh).numpy()

    f1_scores = {}
    for i, v in enumerate(value_classes):
        f1_scores[v] = round(f1_score(y_true[:, i], y_pred[:, i], zero_division=0), 2)

    f1_scores['avg-f1-score'] = round(np.mean(list(f1_scores.values())), 2)

    return f1_scores

In [41]:
def compute_metrics(eval_pred, value_classes):
    """Custom metric calculation function for MultiLabelTrainer"""
    predictions, labels = eval_pred
    f1scores = f1_score_per_label(predictions, labels, value_classes)
    return {'accuracy_thresh': accuracy_thresh(predictions, labels), 'f1-score': f1scores,
            'marco-avg-f1score': f1scores['avg-f1-score']}

In [26]:
def load_model_from_data_dir(model_dir, num_labels):
    """Loads Bert model from specified directory and converts to CUDA model if available"""
    model = AutoModelForSequenceClassification.from_pretrained(model_dir, num_labels=num_labels)
    if torch.cuda.is_available():
        return model.to('cuda')
    return model

In [27]:
class MultiLabelTrainer(Trainer):
    """
        A transformers `Trainer` with custom loss computation
        Methods
        -------
        compute_loss(model, inputs, return_outputs=False):
            Overrides loss computation from Trainer class
        """
    def compute_loss(self, model, inputs, return_outputs=False):
        """Custom loss computation"""
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.BCEWithLogitsLoss()
        loss = loss_fct(logits.view(-1, self.model.config.num_labels),
                        labels.float().view(-1, self.model.config.num_labels))
        return (loss, outputs) if return_outputs else loss

In [28]:
def train_bert_model(train_dataframe, model_dir, labels, test_dataframe=None, num_train_epochs=20):
    """
        Trains Bert model with the arguments in `train_dataframe`
        Parameters
        ----------
        train_dataframe: pd.DataFrame
            The arguments to be trained on
        model_dir: str
            The directory for storing the trained model
        labels : list[str]
            The labels in the training data
        test_dataframe: pd.DataFrame, optional
            The validation arguments (default is None)
        num_train_epochs: int, optional
            The number of training epochs (default is 20)
        Returns
        -------
        Metrics
            result of validation if `test_dataframe` is not None
        NoneType
            otherwise
        """
    if test_dataframe is None:
        test_dataframe = train_dataframe
    ds, labels = convert_to_dataset(train_dataframe, test_dataframe, labels)

    batch_size = 8

    args = TrainingArguments(
        output_dir=model_dir,
        evaluation_strategy="steps",
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_train_epochs,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model='marco-avg-f1score'
    )

    model = load_model_from_data_dir("bert-base-uncased", num_labels=len(labels))

    multi_trainer = MultiLabelTrainer(
        model,
        args,
        train_dataset=ds["train"],
        eval_dataset=ds["test"],
        compute_metrics=lambda x: compute_metrics(x, labels),
        tokenizer=tokenizer
    )

    multi_trainer.train()

    model.save_pretrained(model_dir)

In [32]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 12.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [33]:
from datasets import (Dataset, DatasetDict, load_dataset)

In [30]:
def convert_to_dataset(train_dataframe, test_dataframe, labels):
    """
        Converts pandas DataFrames into a DatasetDict
        Parameters
        ----------
        train_dataframe : pd.DataFrame
            Arguments to be listed as "train"
        test_dataframe : pd.DataFrame
            Arguments to be listed as "test"
        labels : list[str]
            The labels in both DataFrames
        Returns
        -------
        tuple(DatasetDict, list[str])
            a `DatasetDict` with attributes "train" and "test" for the listed arguments,
            a `list` with the contained labels
        """
    column_intersect = [x for x in (['Premise'] + labels) if x in train_dataframe.columns.values]

    train_dataset = Dataset.from_dict((train_dataframe[column_intersect]).to_dict('list'))
    test_dataset = Dataset.from_dict((test_dataframe[column_intersect]).to_dict('list'))

    ds = DatasetDict()
    ds['train'] = train_dataset
    ds['test'] = test_dataset

    ds = ds.map(lambda x: {"labels": [int(x[c]) for c in ds['train'].column_names if
                                      c not in ['Argument ID', 'Conclusion', 'Stance', 'Premise', 'Part']]})

    cols = ds['train'].column_names
    cols.remove('labels')

    ds_enc = ds.map(tokenize_and_encode, batched=True, remove_columns=cols)

    cols.remove('Premise')

    return ds_enc, cols

In [36]:
def tokenize_and_encode(examples):
    """Tokenizes each arguments "Premise" """
    return tokenizer(examples['Premise'], truncation=True)

In [42]:
validate = False
for i in range(num_levels):
            print("===> Bert: Training Level %s..." % levels[i])
            if validate:
                bert_model_evaluation = train_bert_model(df_train_all[i],
                                                         os.path.join('bert_train_level{}'.format(levels[i])),
                                                         values[levels[i]], test_dataframe=df_valid_all[i])
                print("F1-Scores for Level %s:" % levels[i])
                print(bert_model_evaluation['eval_f1-score'])
            else:
                train_bert_model(df_train_all[i], os.path.join('bert_train_level{}'.format(levels[i])),
                                 values[levels[i]])

===> Bert: Training Level 1...


  0%|          | 0/5270 [00:00<?, ?ex/s]

  0%|          | 0/5270 [00:00<?, ?ex/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

using `logging_steps` to initialize `eval_steps` to 500
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8"

Step,Training Loss,Validation Loss,Accuracy Thresh,F1-score,Marco-avg-f1score
500,0.297700,0.246926,0.920356,"{'Be ambitious': 0.0, 'Be behaving properly': 0.0, 'Be broadminded': 0.0, 'Be capable': 0.0, 'Be choosing own goals': 0.0, 'Be compliant': 0.0, 'Be courageous': 0.0, 'Be creative': 0.0, 'Be curious': 0.0, 'Be daring': 0.0, 'Be forgiving': 0.0, 'Be helpful': 0.0, 'Be holding religious faith': 0.0, 'Be honest': 0.0, 'Be honoring elders': 0.0, 'Be humble': 0.0, 'Be independent': 0.0, 'Be intellectual': 0.0, 'Be just': 0.0, 'Be logical': 0.0, 'Be loving': 0.0, 'Be neat and tidy': 0.0, 'Be polite': 0.0, 'Be protecting the environment': 0.0, 'Be respecting traditions': 0.0, 'Be responsible': 0.0, 'Be self-disciplined': 0.0, 'Have a comfortable life': 0.0, 'Have a good reputation': 0.0, 'Have a safe country': 0.0, 'Have a sense of belonging': 0.0, 'Have a stable society': 0.0, 'Have a varied life': 0.0, 'Have a world at peace': 0.0, 'Have a world of beauty': 0.0, 'Have an exciting life': 0.0, 'Have an objective view': 0.0, 'Have equality': 0.0, 'Have freedom of action': 0.0, 'Have freedom of thought': 0.0, 'Have good health': 0.0, 'Have harmony with nature': 0.0, 'Have influence': 0.0, 'Have life accepted as is': 0.0, 'Have loyalty towards friends': 0.0, 'Have no debts': 0.0, 'Have pleasure': 0.0, 'Have privacy': 0.0, 'Have social recognition': 0.0, 'Have success': 0.0, 'Have the own family secured': 0.0, 'Have the right to command': 0.0, 'Have the wisdom to accept others': 0.0, 'Have wealth': 0.0, 'avg-f1-score': 0.0}",0.000000
1000,0.236300,0.216935,0.924454,"{'Be ambitious': 0.0, 'Be behaving properly': 0.0, 'Be broadminded': 0.0, 'Be capable': 0.0, 'Be choosing own goals': 0.05, 'Be compliant': 0.0, 'Be courageous': 0.0, 'Be creative': 0.0, 'Be curious': 0.0, 'Be daring': 0.0, 'Be forgiving': 0.0, 'Be helpful': 0.0, 'Be holding religious faith': 0.0, 'Be honest': 0.0, 'Be honoring elders': 0.0, 'Be humble': 0.0, 'Be independent': 0.0, 'Be intellectual': 0.0, 'Be just': 0.37, 'Be logical': 0.0, 'Be loving': 0.0, 'Be neat and tidy': 0.0, 'Be polite': 0.0, 'Be protecting the environment': 0.0, 'Be respecting traditions': 0.0, 'Be responsible': 0.0, 'Be self-disciplined': 0.0, 'Have a comfortable life': 0.0, 'Have a good reputation': 0.0, 'Have a safe country': 0.63, 'Have a sense of belonging': 0.0, 'Have a stable society': 0.24, 'Have a varied life': 0.0, 'Have a world at peace': 0.19, 'Have a world of beauty': 0.0, 'Have an exciting life': 0.0, 'Have an objective view': 0.0, 'Have equality': 0.1, 'Have freedom of action': 0.42, 'Have freedom of thought': 0.05, 'Have good health': 0.44, 'Have harmony with nature': 0.0, 'Have influence': 0.0, 'Have life accepted as is': 0.0, 'Have loyalty towards friends': 0.0, 'Have no debts': 0.0, 'Have pleasure': 0.0, 'Have privacy': 0.0, 'Have social recognition': 0.0, 'Have success': 0.0, 'Have the own family secured': 0.0, 'Have the right to command': 0.0, 'Have the wisdom to accept others': 0.0, 'Have wealth': 0.0, 'avg-f1-score': 0.05}",0.050000


***** Running Evaluation *****
  Num examples = 5270
  Batch size = 8
Trainer is attempting to log a value of "{'Be ambitious': 0.0, 'Be behaving properly': 0.0, 'Be broadminded': 0.0, 'Be capable': 0.0, 'Be choosing own goals': 0.0, 'Be compliant': 0.0, 'Be courageous': 0.0, 'Be creative': 0.0, 'Be curious': 0.0, 'Be daring': 0.0, 'Be forgiving': 0.0, 'Be helpful': 0.0, 'Be holding religious faith': 0.0, 'Be honest': 0.0, 'Be honoring elders': 0.0, 'Be humble': 0.0, 'Be independent': 0.0, 'Be intellectual': 0.0, 'Be just': 0.0, 'Be logical': 0.0, 'Be loving': 0.0, 'Be neat and tidy': 0.0, 'Be polite': 0.0, 'Be protecting the environment': 0.0, 'Be respecting traditions': 0.0, 'Be responsible': 0.0, 'Be self-disciplined': 0.0, 'Have a comfortable life': 0.0, 'Have a good reputation': 0.0, 'Have a safe country': 0.0, 'Have a sense of belonging': 0.0, 'Have a stable society': 0.0, 'Have a varied life': 0.0, 'Have a world at peace': 0.0, 'Have a world of beauty': 0.0, 'Have an exciting li

Step,Training Loss,Validation Loss,Accuracy Thresh,F1-score,Marco-avg-f1score
500,0.297700,0.246926,0.920356,"{'Be ambitious': 0.0, 'Be behaving properly': 0.0, 'Be broadminded': 0.0, 'Be capable': 0.0, 'Be choosing own goals': 0.0, 'Be compliant': 0.0, 'Be courageous': 0.0, 'Be creative': 0.0, 'Be curious': 0.0, 'Be daring': 0.0, 'Be forgiving': 0.0, 'Be helpful': 0.0, 'Be holding religious faith': 0.0, 'Be honest': 0.0, 'Be honoring elders': 0.0, 'Be humble': 0.0, 'Be independent': 0.0, 'Be intellectual': 0.0, 'Be just': 0.0, 'Be logical': 0.0, 'Be loving': 0.0, 'Be neat and tidy': 0.0, 'Be polite': 0.0, 'Be protecting the environment': 0.0, 'Be respecting traditions': 0.0, 'Be responsible': 0.0, 'Be self-disciplined': 0.0, 'Have a comfortable life': 0.0, 'Have a good reputation': 0.0, 'Have a safe country': 0.0, 'Have a sense of belonging': 0.0, 'Have a stable society': 0.0, 'Have a varied life': 0.0, 'Have a world at peace': 0.0, 'Have a world of beauty': 0.0, 'Have an exciting life': 0.0, 'Have an objective view': 0.0, 'Have equality': 0.0, 'Have freedom of action': 0.0, 'Have freedom of thought': 0.0, 'Have good health': 0.0, 'Have harmony with nature': 0.0, 'Have influence': 0.0, 'Have life accepted as is': 0.0, 'Have loyalty towards friends': 0.0, 'Have no debts': 0.0, 'Have pleasure': 0.0, 'Have privacy': 0.0, 'Have social recognition': 0.0, 'Have success': 0.0, 'Have the own family secured': 0.0, 'Have the right to command': 0.0, 'Have the wisdom to accept others': 0.0, 'Have wealth': 0.0, 'avg-f1-score': 0.0}",0.000000
1000,0.236300,0.216935,0.924454,"{'Be ambitious': 0.0, 'Be behaving properly': 0.0, 'Be broadminded': 0.0, 'Be capable': 0.0, 'Be choosing own goals': 0.05, 'Be compliant': 0.0, 'Be courageous': 0.0, 'Be creative': 0.0, 'Be curious': 0.0, 'Be daring': 0.0, 'Be forgiving': 0.0, 'Be helpful': 0.0, 'Be holding religious faith': 0.0, 'Be honest': 0.0, 'Be honoring elders': 0.0, 'Be humble': 0.0, 'Be independent': 0.0, 'Be intellectual': 0.0, 'Be just': 0.37, 'Be logical': 0.0, 'Be loving': 0.0, 'Be neat and tidy': 0.0, 'Be polite': 0.0, 'Be protecting the environment': 0.0, 'Be respecting traditions': 0.0, 'Be responsible': 0.0, 'Be self-disciplined': 0.0, 'Have a comfortable life': 0.0, 'Have a good reputation': 0.0, 'Have a safe country': 0.63, 'Have a sense of belonging': 0.0, 'Have a stable society': 0.24, 'Have a varied life': 0.0, 'Have a world at peace': 0.19, 'Have a world of beauty': 0.0, 'Have an exciting life': 0.0, 'Have an objective view': 0.0, 'Have equality': 0.1, 'Have freedom of action': 0.42, 'Have freedom of thought': 0.05, 'Have good health': 0.44, 'Have harmony with nature': 0.0, 'Have influence': 0.0, 'Have life accepted as is': 0.0, 'Have loyalty towards friends': 0.0, 'Have no debts': 0.0, 'Have pleasure': 0.0, 'Have privacy': 0.0, 'Have social recognition': 0.0, 'Have success': 0.0, 'Have the own family secured': 0.0, 'Have the right to command': 0.0, 'Have the wisdom to accept others': 0.0, 'Have wealth': 0.0, 'avg-f1-score': 0.05}",0.050000


KeyboardInterrupt: ignored